# Dream for Colab

Original repo: [dreamnettech/dreampower](https://github.com/dreamnettech/dreampower/tree/v1.2.10)

Colab: [FirstDee/dreamtime-v1.ipynb](https://gist.github.com/FirstDee/c3a94ff37e0561e597b65160aabbbc07)

This colab only points out how the output resolution can be configured and compressed the amount of cells. Thanks to FirstDee for creating the Colab.

In [ ]:
#@title Check GPU
!nvidia-smi

# Project Setup
v1.2.10 contains --image-size, which can be applied in the last cell, but the output tends to look bad. Automatic mask creation seems to be pretty bad with this resolution. Only use this if you want to experiment. You may need to change overlay_size as well.

It may take a few minutes to load the program. Please be patient.

In [ ]:
#@title Project Settings

import os

#@markdown Repo path to clone from.
repo_url = 'https://github.com/dreamnettech/dreampower' #@param {type:"string"}
repo_name = repo_url.split('/')[-1]

#@markdown Repo branch or tag to clone. Leave blank to clone `master`.
repo_branch = 'v1.2.10' #@param {type:"string"}

#@markdown Root workspace.
root_dir = '/content/dreampower' #@param {type:"string"}
repo_dir = os.path.join(root_dir, repo_name)

#@markdown Temp directory. If your root workspace is in Google Drive, then this path should be a local folder, such as `/content`.
temp_dir = '/content/dreampower/work' #@param {type:"string"}

#@markdown Models directory.
model_dir = '/content/dreampower/work/checkpoint' #@param {type:"string"}

#@markdown Masks directory.
altered_dir = '/content/dreampower/work/altered' #@param {type:"string"}

#@markdown Output directory.
output_dir = '/content/dreampower/work/output' #@param {type:"string"}

#@markdown Attempt to request Google Drive access. Will mount to `/content/drive/My Drive`. 
#@markdown
#@markdown Try auto-mounting by clicking the Files button on the left, then click `Mount Drive`.
request_drive = False #@param {type:"boolean"} 

#@markdown Install `dependencies.txt` from project.
install_deps = True #@param {type:"boolean"}

#@#markdown Tensorflow version (1 or 2)
tf_version = 1 #@#param {type:"integer"}

#@#markdown Attempt to invoke high memory (25.5 GB) in Colab.
high_memory = False #@#param {type:"boolean"}


In [ ]:
#@title Setup

!pip install gputil

# Import packages
import os,sys,humanize,psutil,GPUtil

def mem_report():
  print("CPU RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ))
  
  GPUs = GPUtil.getGPUs()
  for i, gpu in enumerate(GPUs):
    print('GPU {:d} ... Mem Free: {:.0f}MB / {:.0f}MB | Utilization {:3.0f}%'.format(i, gpu.memoryFree, gpu.memoryTotal, gpu.memoryUtil*100))
    
mem_report()

if high_memory and psutil.virtual_memory().available < 13958643712:
  print('Attempting to invoke high memory.')
  print('This notebook will crash intentionally and Colab should display a prompt to offer you high-RAM.')
  print('IF THIS PROMPT DOES NOT SHOW, DISABLE high_memory IN THE SETTINGS!')
  d=[]
  while(1):
    d.append('1')

# Trigger tensorflow version
if tf_version == 2:
  %tensorflow_version 2.x
else:
  %tensorflow_version 1.x

from google.colab import drive

if request_drive:
  drive.mount('/content/drive')

os.makedirs(root_dir, exist_ok=True)
os.makedirs(temp_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)
os.makedirs(altered_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

if not os.path.isdir(repo_dir):
  repo_cmd = f'--branch {repo_branch}' if repo_branch else ''
  !git clone {repo_url} {repo_cmd} --depth 1 "{repo_dir}" 

  # Code hack (Commented out because unnecessary): 
  # We manage our own mask (altered) directory tree,
  # so prevent dreampower from adding a hash to its altered paths.
  # However, this errors for `n_runs` > 1, so add a check.
  # See processing/image.py line 47, v1.2.5
  #!sed 's/self.__altered_path = folder_path/if Conf.args["n_runs"] > 1: self.__altered_path = folder_path/' -i "{repo_dir}/processing/image.py"

# https://github.com/dreamnettech/dreampower-checkpoints/releases

if not os.path.exists(os.path.join(model_dir, 'checkpoints')):
  %cd "{temp_dir}"
  if not os.path.exists(os.path.join(temp_dir, 'model.zip')):
    !wget -O ./model.zip https://github.com/dreamnettech/dreampower-checkpoints/releases/download/0.0.1/v0.0.1.zip
  !unzip ./model.zip -d "{model_dir}"


if install_deps:
  #%cd "{root_dir}/{repo_name}"
  %cd /content/dreampower/dreampower/src

  # Strip tensorflow from requirements list because Colab provides this custom-compiled.
  !sed '/tensorflow/d' -i requirements.txt

  !pip install -r requirements.txt

# Upload Images for Processing

In [ ]:
#@title Upload Images for Processing

from IPython.core.display import display, HTML
from IPython.display import Image

from google.colab import files
import shutil

input_dir = os.path.join(temp_dir, 'inputs')
if os.path.exists(input_dir):
  shutil.rmtree(input_dir)
os.makedirs(input_dir)

%cd "{temp_dir}"

inputs = files.upload()

for i, fn in enumerate(inputs.keys()):
  with open(os.path.join(input_dir, fn), 'wb') as f:
    f.write(bytearray(inputs[fn]))
  if i < 5:
    display(Image(os.path.join(input_dir, fn), width=512))

if len(inputs) > 5:
    print('Displayed the first 5 images for processing.')

# Upload Custom Mask (Advanced)

In [ ]:
#@title Upload Custom Mask

#@markdown You may upload a custom mask during runtime by checking the box below. This applies only if you upload one input image.
use_custom_maskfin = False #@param {type:"boolean"}

#@markdown The mask must be in PNG format in 512x512 size. [See this image for examples of each mask step](https://github.com/dreamnettech/dreampower/blob/v1.2.5/assets/transformation.jpg).
#@markdown
#@markdown When drawing your own mask, consider starting from an existing mask step, such as `DressToCorrect.png`. Run your input image normally, then look in the `/content/dreampower/work/altered` directory for existing masks.

from google.colab import files
import hashlib

#@markdown If you wish to use existing image steps, specify the `altered` directory here:
custom_altered_dir = '' #@param {type:'string'}
#@markdown Specify the step to begin processing. Steps with corresponding images:
#@markdown ```
#@markdown - :                    [ImageToCrop.png]
#@markdown - :                    [ImageToResized.png]
#@markdown 0 : dress -> correct   [DressToCorrect.png]
#@markdown 1 : correct -> mask    [CorrectToMask.png]
#@markdown 2 : mask -> maskref    [MaskToMaskref.png]
#@markdown 3 : maskref -> maskdet [MaskrefToMaskdet.png]
#@markdown 4 : maskdet -> maskfin [MaskdetToMaskfin.png]
#@markdown 5 : maskfin -> nude    [MaskfinToNude.png]
#@markdown - :                    [ImageToOverlay.png]
#@markdown ```
#@markdown These settings apply only if you are uploading a custom mask.
starting_step = 5 #@param {type:"integer"}
ending_step = 5 #@param {type:"integer"}
mask_n_runs = 1 #@param {type:"integer"}

input_mask_dir = ''

step_names = ['DressToCorrect.png','CorrectToMask.png','MaskToMaskref.png','MaskrefToMaskdet.png','MaskdetToMaskfin.png','MaskfinToNude.png']

if use_custom_maskfin:
  if starting_step < 1:
    raise('Must use custom mask step between 1 and 5. If using 0, then disable `use_custom_maskfin`.')
    
  if len(os.listdir(input_dir)) != 1:
    raise('Custom masks can only be used with one input image.')

  if os.path.isdir(custom_altered_dir):
    input_mask_dir = custom_altered_dir
  else:
    input_mask_dir = os.path.join(temp_dir, 'masks')
    if os.path.exists(input_mask_dir):
      shutil.rmtree(input_mask_dir)
    os.makedirs(input_mask_dir)

  # Construct altered subdir for input file
  # See processing/image.py line 34 v1.2.5
  input_fn = os.path.join(input_dir, os.listdir(input_dir)[0])
  input_basename = os.path.splitext(os.listdir(input_dir)[0])[0]
  input_hashedname = f'{input_basename}_{str(hashlib.md5(open(input_fn, "rb").read()).hexdigest())}'
  input_mask_subdir = os.path.join(input_mask_dir, input_hashedname)
  os.makedirs(input_mask_subdir, exist_ok=True)

  %cd "{temp_dir}"

  inputs = files.upload()

  for i, fn in enumerate(inputs.keys()):
    if not '.png' in fn.lower():
      raise('Mask must be in PNG format.')
    with open(os.path.join(input_mask_subdir, step_names[starting_step-1]), 'wb') as f:
      f.write(bytearray(inputs[fn]))
    
    if i < 5:
      display(Image(os.path.join(input_mask_subdir, step_names[starting_step-1]), width=512))

  if len(inputs) > 5:
      print('Displayed the first 5 images for processing.')

  # HACK: Copy Maskfin to the other step images, because dreampower expects them.
  # This has no effect on the MaskfinToNude step.
  if not os.path.isdir(custom_altered_dir):
    %cd "{input_mask_subdir}"
    shutil.copyfile(step_names[starting_step-1], 'ImageToCrop.png')
    shutil.copyfile(step_names[starting_step-1], 'ImageToResized.png')
    for i in range(0, starting_step-1):
      shutil.copyfile(step_names[starting_step-1], step_names[i])

# Processing

In [ ]:
#@title Processing Settings

#@markdown #### Scaling Mode
#@markdown Choose how the image will be scaled for processing, or apply no scaling.

resize_mode = 'Fit to Square' #@param ['Fit to Square','Crop to Square','Overlay','None']
resize_mode = resize_mode.split(' ')[0]

#@markdown If selecting `Overlay`, then specify the top left corner and width of the processing region.

overlay_x = 0 #@param {type:'integer'}
overlay_y = 0 #@param {type:'integer'}
overlay_size = 512 #@param {type:'integer'}
overlay_dims = f'{overlay_x},{overlay_y}:{overlay_x+overlay_size},{overlay_y+overlay_size}'

#@markdown Attempt to recover original colors into the output image.
color_transfer = False #@param {type:'boolean'}

#@markdown #### Alteration Modes
#@markdown **NOTE: If you uploaded a custom mask above, then these settings have no effect!**
#@markdown * **Manual**: Specify exact sizes manually.
#@markdown * **Randomize**: Randomize the sizes by `step` over `n_runs`. `min` and `max` affect the randomization range.
#@markdown * **Increase by Step**: Increase each size by `step` over `n_runs`.
#@markdown
#@markdown Setting each `step` to `0` will disable the randomization or increase for that body part. 
#@markdown
#@markdown In Manual mode, the `step` setting has no effect.

runs_mode = 'Manual' #@param ['Manual', 'Randomize', 'Increase by Step']
runs_mode = runs_mode.split(' ')[0]
n_runs =  1#@param {type:"integer"}
n_runs = max(1, n_runs)
#@markdown ---
boob_size = 1 #@param {type:"slider", min:0.3, max:2.0, step:0.05}
boob_step = 0.1 #@param {type:"slider", min:0.0, max:2, step:0.05}
boob_min = 0.3 #@param {type:"slider", min:0.3, max:2.0, step:0.05}
boob_max = 2.0 #@param {type:"slider", min:0.3, max:2.0, step:0.05}
#@markdown ---
areola_size = 1 #@param {type:"slider", min:0.3, max:2.0, step:0.05}
areola_step = 0.1 #@param {type:"slider", min:0.0, max:2, step:0.05}
areola_min = 0.3 #@param {type:"slider", min:0.3, max:2.0, step:0.05}
areola_max = 2.0 #@param {type:"slider", min:0.3, max:2.0, step:0.05}
#@markdown ---
nipple_size = 1 #@param {type:"slider", min:0.3, max:2.0, step:0.05}
nipple_step = 0.1 #@param {type:"slider", min:0.0, max:2, step:0.05}
nipple_min = 0.3 #@param {type:"slider", min:0.3, max:2.0, step:0.05}
nipple_max = 2.0 #@param {type:"slider", min:0.3, max:2.0, step:0.05}
#@markdown ---
vagina_size = 0.75 #@param {type:"slider", min:0.3, max:1.5, step:0.05}
vagina_step = 0.1 #@param {type:"slider", min:0.0, max:1.5, step:0.05}
vagina_min = 0.3 #@param {type:"slider", min:0.3, max:1.5, step:0.05}
vagina_max = 1.5 #@param {type:"slider", min:0.3, max:1.5, step:0.05}
#@markdown ---
hair_size = 1 #@param {type:"slider", min:0.3, max:2.0, step:0.05}
hair_step = 0.1 #@param {type:"slider", min:0.0, max:2.0, step:0.05}
hair_min = 0.3 #@param {type:"slider", min:0.3, max:2.0, step:0.05}
hair_max = 2 #@param {type:"slider", min:0.3, max:2.0, step:0.05}
#@markdown ---

#@markdown List size parameters in the output filename.
size_names = False #@param {type:'boolean'}

In [ ]:
#@title Generate Images

from datetime import datetime

# Build base params
checkpoints = os.path.join(model_dir, 'checkpoints')
cmd_params = ['run',
              '-c',f'"{checkpoints}"']

if resize_mode == 'Overlay':
  cmd_params += ['--overlay',overlay_dims]
elif resize_mode == 'Crop':
  cmd_params.append('--auto-resize-crop')
elif resize_mode == 'Fit':
  cmd_params.append('--auto-resize')

if color_transfer:
  cmd_params.append('--color-transfer')

#cmd_params.append('--image-size 1024')

cmd_string = ' '.join(cmd_params)

# Build file list

files = []

def is_image(f):
  return f.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif'))

for fn in os.listdir(input_dir):
  if is_image(fn):
    files.append(fn)

# Build jobs

jobs, job_names = [], []
num_f = '{:.2f}'

if use_custom_maskfin:
  jobs.append(['--n-runs',str(mask_n_runs)])
  job_names.append('')
elif runs_mode == 'Randomize':
  import random
  for n in range(0, n_runs):
    bmin, amin, nmin, vmin, hmin = (
        min(boob_min, boob_max),
        min(areola_min, areola_max),
        min(nipple_min, nipple_max),
        min(vagina_min, vagina_max),
        min(hair_min, hair_max)
    )
    bmax, amax, nmax, vmax, hmax = (
        max(boob_min, boob_max),
        max(areola_min, areola_max),
        max(nipple_min, nipple_max),
        max(vagina_min, vagina_max),
        max(hair_min, hair_max)
    )
    bsize, asize, nsize, vsize, hsize = (
        (random.randrange(bmin*100, bmax*100, (boob_step*100))/100) if boob_step else boob_size,
        (random.randrange(amin*100, amax*100, (areola_step*100))/100) if areola_step else areola_size,
        (random.randrange(nmin*100, nmax*100, (nipple_step*100))/100) if nipple_step else nipple_size,
        (random.randrange(vmin*100, vmax*100, (vagina_step*100))/100) if vagina_step else vagina_size,
        (random.randrange(hmin*100, hmax*100, (hair_step*100))/100) if hair_step else hair_size
    )
    jobs.append([str(x) for x in ['--n-runs','1',
                                  '--bsize',bsize,
                                  '--asize',asize,
                                  '--nsize',nsize,
                                  '--vsize',vsize,
                                  '--hsize',hsize]])
    job_names.append(f'b-{num_f.format(bsize)}_a-{num_f.format(asize)}_n-{num_f.format(nsize)}_v-{num_f.format(vsize)}_h-{num_f.format(hsize)}')
elif runs_mode == 'Increase' and n_runs > 1:
  for n in range(0, n_runs):
    bsize, asize, nsize, vsize, hsize = (
        min(2.0, boob_size+(boob_step*n)),
        min(2.0, areola_size+(areola_step*n)),
        min(2.0, nipple_size+(nipple_step*n)),
        min(1.5, vagina_size+(vagina_step*n)),
        min(2.0, hair_size+(hair_step*n))
    )
    jobs.append([str(x) for x in ['--n-runs','1',
                                  '--bsize',bsize,
                                  '--asize',asize,
                                  '--nsize',nsize,
                                  '--vsize',vsize,
                                  '--hsize',hsize]])
    job_names.append(f'b-{num_f.format(bsize)}_a-{num_f.format(asize)}_n-{num_f.format(nsize)}_v-{num_f.format(vsize)}_h-{num_f.format(hsize)}')
else:
  jobs.append([str(x) for x in ['--n-runs',n_runs,
                                '--bsize',boob_size,
                                '--asize',areola_size,
                                '--nsize',nipple_size,
                                '--vsize',vagina_size,
                                '--hsize',hair_size]])
  job_names.append(f'b-{num_f.format(boob_size)}_a-{num_f.format(areola_size)}_n-{num_f.format(nipple_size)}_v-{num_f.format(vagina_size)}_h-{num_f.format(hair_size)}')

display(HTML(f'<h1>Running {len(jobs)*len(files)} Jobs</h1><p>See <code>{output_dir}</code> for generated images.</p>'))

timestamp = datetime.now().strftime("%Y%m%d%H%M%S")

for fn in files:
  fn_base = os.path.splitext(fn)[0]
  fn_ext = os.path.splitext(fn)[-1]
  for i, job in enumerate(jobs):
    output_basename = f'{fn_base}_{timestamp}'
    output_basename_jobnum = f'{(i+1):03}'
    output_basename_size = f'_{job_names[i]}'    
    output_fn = f'{output_basename}-{output_basename_jobnum}{output_basename_size if size_names else ""}{fn_ext}'
    output_path = os.path.join(output_dir,output_fn)

    if use_custom_maskfin:
      altered_path = input_mask_dir
      mask_string = f'--steps {starting_step}:{ending_step}'
    else:
      output_alteredname = f'{output_basename_jobnum}{output_basename_size}'
      altered_path = os.path.join(altered_dir,output_basename,output_alteredname)
      mask_string = ''

    file_string = ' '.join(['--input',f'"{os.path.join(input_dir,fn)}"',
                            '--output',f'"{output_path}"',
                            '--altered',f'"{altered_path}"'])

    job_string = ' '.join(job)

    display(HTML(f'<h1>Job {i+1}</h1><p>Output: <code>{output_path}</code></p><p>Masks: <code>{altered_path}</code></p><p><code>{job_string}</code></p>'))

    !python3 "/content/dreampower/dreampower/src/main.py" {cmd_string} {file_string} {job_string} {mask_string}

    if os.path.exists(output_path):
      display(Image(output_path, width=512))